# Set up

In [1]:
# update path with data dir
import sys
sys.path.append('../data/')

In [2]:
import pandas as pd
import knapsack
import cvxpy as cp
import numpy as np
import requests

# Toy 0-1 one-dimensional knapsack example

In [ ]:
o_values = np.array([22, 12, 16, 10, 35, 26, 42, 53])
o_weights = np.array([21, 11, 15, 9, 34, 25, 41, 52])
o_capacity = 100

## `knapsack`

In [ ]:
# Maximize sum of selected value. Sum of selected weight is less than capacity

total_value, items_idx = knapsack.knapsack(o_weights, o_values).solve(o_capacity)
items = [o_values[i] for i in items_idx]

print(sum(items))
print(items)

## `cvxpy`

In [ ]:
o_x = cp.Variable(8, boolean=True)

In [ ]:
o_prob = cp.Problem(
    cp.Maximize(o_values@o_x),
    [o_weights@o_x <= o_capacity]
)

In [ ]:
o_prob.solve()

In [ ]:
o_selection = [int(round(j)) for j in o_x.value]
o_selection

# Toy 0-1 multidimensional knapsack example

In [ ]:
m_values = np.array([22, 12, 16, 10, 35, 26, 42, 53])
m_weights = np.array([
    [21, 11, 15, 9, 34, 25, 41, 52],
    [28, 8, 25, 3, 34, 25, 35, 60]
])
m_capacity = np.array([100, 125])
m_num = 3

In [ ]:
m_x = cp.Variable(8, boolean=True)

In [ ]:
m_prob = cp.Problem(
    cp.Maximize(m_values@m_x),
    [
        m_weights@m_x <= m_capacity,
        np.ones(8)@m_x == m_num
    ]
)

In [ ]:
m_prob.solve()

In [ ]:
m_selection = [int(round(j)) for j in m_x.value]
m_selection

# Toy team optimisation

In [274]:
# bootstrap_static_d_last_season = bootstrap_static_d

In [310]:
bootstrap_static_request = requests.get('https://fantasy.premierleague.com/api/bootstrap-static/')
bootstrap_static_d = bootstrap_static_request.json()

In [311]:
black_list_players = [
#     'David Luiz',
#     'Pogba',
#     'Alonso',
#     'Alexander-Arnold',
#     'Azpilicueta',
#     'Kanté',
]
players = [i for i in bootstrap_static_d['elements'] if i['web_name'] not in black_list_players]

In [312]:
player_names = np.array([i['web_name'] for i in players])
player_points = np.array([i['total_points'] for i in players])/38
player_costs = np.array([[i['now_cost'] for i in players]])
player_position = np.array([i['element_type'] for i in players])
player_team = np.array([i['team'] for i in players])

In [313]:
player_position_weights = np.zeros((4, len(players)))
for i in range(0, 4):
    for j in range(0, len(players)):
        if player_position[j] == i+1:
            player_position_weights[i, j] = 1
        else:
            player_position_weights[i, j] = 0

In [314]:
player_team_weights = np.zeros((20, len(players)))
for i in range(0, 20):
    for j in range(0, len(players)):
        if player_team[j] == i+1:
            player_team_weights[i, j] = 1
        else:
            player_team_weights[i, j] = 0

In [367]:
player_position_capacity = [1, 3, 4, 3]
bench_position_capacity = [1, 2, 1, 0]

In [368]:
min_cost_keeper = np.min([i['now_cost'] for i in players if i['element_type'] == 1])
min_cost_defender = np.min([i['now_cost'] for i in players if i['element_type'] == 2])
min_cost_midfielder = np.min([i['now_cost'] for i in players if i['element_type'] == 3])
min_cost_striker = np.min([i['now_cost'] for i in players if i['element_type'] == 4])
min_cost_bench = \
bench_position_capacity@np.array([min_cost_keeper, min_cost_defender, min_cost_midfielder, min_cost_striker])

In [369]:
player_cost_capacity = [997 - min_cost_bench]
player_team_capacity = [3]*20

In [370]:
player_weights = np.concatenate((
    player_costs,
    player_position_weights,
    player_team_weights
), axis=0)
player_capacity = np.array(
    player_cost_capacity
    + player_position_capacity
    + player_team_capacity
)

player_num = 11

In [371]:
bench_num = 4

## From scratch

### Normal

In [372]:
fs_player_x = cp.Variable(len(players), boolean=True)

In [373]:
fs_player_prob = cp.Problem(
    cp.Maximize(player_points@fs_player_x),
    [
        player_weights@fs_player_x <= player_capacity,
        np.ones(len(players))@fs_player_x == player_num
    ]
)

In [337]:
# 1, 5, 4, 1
fs_player_prob.solve()

7.868421052631764

In [355]:
# 1, 4, 5, 1
fs_player_prob.solve()

7.947368421052791

In [366]:
# 1, 4, 4, 2
fs_player_prob.solve()

8.131578947022035

In [374]:
# 1, 3, 4, 3
fs_player_prob.solve()

8.18421052631651

In [356]:
fs_player_prob.status

'optimal'

In [375]:
fs_player_selection = [int(round(j)) for j in fs_player_x.value]
[i for i, j in zip(player_names, fs_player_selection) if j == 1]

['Barnes',
 'Abraham',
 'van Aanholt',
 'Digne',
 'Salah',
 'Sterling',
 'De Bruyne',
 'Pukki',
 'Cantwell',
 'Vestergaard',
 'Patrício']

In [340]:
fs_bench_team_capacity = player_team_capacity - player_team_weights@fs_player_selection

In [341]:
fs_bench_capacity = np.array(
    [min_cost_bench]
    + bench_position_capacity
    + list(fs_bench_team_capacity)
)

In [342]:
fs_bench_x = cp.Variable(len(players), boolean=True)

In [343]:
fs_bench_prob = cp.Problem(
    cp.Maximize(player_points@fs_bench_x),
    [
        player_weights@fs_bench_x <= fs_bench_capacity,
        np.ones(len(players))@fs_bench_x == bench_num
    ]
)

In [344]:
fs_bench_prob.solve()

0.0263157894742622

In [345]:
fs_bench_selection = [int(round(j)) for j in fs_bench_x.value]
[i for i, j in zip(player_names, fs_bench_selection) if j == 1]

['Nketiah', 'Lansbury', 'Surridge', 'Woodman']

In [346]:
abs(round(1000 - (player_costs@fs_player_x.value)[0] - min_cost_bench))

8.0

### Captain

In [100]:
cap_player_x = cp.Variable(len(players), integer=True)

In [103]:
cap_player_prob = cp.Problem(
    cp.Maximize(player_points@cap_player_x),
    [
        player_weights@cap_player_x <= player_capacity,
        np.ones(len(players))@cap_player_x == 12,
        cap_player_x@(cap_player_x ) == 14
    ]
)

In [104]:
cap_player_prob.solve()

1918128470.4503849

In [105]:
cap_player_selection = [int(round(j)) for j in cap_player_x.value]
[i for i, j in zip(player_names, cap_player_selection) if j == 1]

[]

In [99]:
cap_selection = [int(round(j)) for j in cap_player_y.value]
[i for i, j in zip(player_names, cap_selection) if j == 1]

['Salah']

In [17]:
cap_bench_team_capacity = player_team_capacity - player_team_weights@cap_player_selection

In [18]:
cap_bench_capacity = np.array(
    [min_cost_bench]
    + bench_position_capacity
    + list(cap_bench_team_capacity)
)

In [19]:
cap_bench_x = cp.Variable(len(players), boolean=True)

In [20]:
cap_bench_prob = cp.Problem(
    cp.Maximize(player_points@cap_bench_x),
    [
        player_weights@cap_bench_x <= cap_bench_capacity,
        np.ones(len(players))@cap_bench_x == bench_num
    ]
)

In [21]:
cap_bench_prob.solve()

1.5789473685256235

In [22]:
cap_bench_selection = [int(round(j)) for j in cap_bench_x.value]
[i for i, j in zip(player_names, cap_bench_selection) if j == 1]

['Nketiah', 'Button', 'Kelly', 'Wickham']

In [23]:
abs(round(1000 - (player_costs@cap_player_x.value)[0] - min_cost_bench))

0.0

## Existing team

In [299]:
existing_team_indices = [
list(player_names).index('Ederson'),
list(player_names).index('Robertson'),
list(player_names).index('van Dijk'),
list(player_names).index('Laporte'),
list(player_names).index('Digne'),
list(player_names).index('Wan-Bissaka'),
list(player_names).index('Salah'),
list(player_names).index('Kanté'),
list(player_names).index('Milivojevic'),
list(player_names).index('Sterling'),
list(player_names).index('Jiménez'),
]

In [300]:
et_x_0 = np.array([1 if i in existing_team_indices else 0 for i in range(0, len(player_names))])
et_x_0


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [301]:
et_selection_0 = [int(round(j)) for j in et_x_0]
et_player_selection_0_names = [i for i, j in zip(player_names, et_selection_0) if j == 1]
et_player_selection_0_names

['Kanté',
 'Milivojevic',
 'Digne',
 'Robertson',
 'van Dijk',
 'Salah',
 'Laporte',
 'Ederson',
 'Sterling',
 'Wan-Bissaka',
 'Jiménez']

In [302]:
et_player_x = cp.Variable(len(players), boolean=True)

In [307]:
et_player_prob = cp.Problem(
    cp.Maximize(
        player_points@et_player_x - 2*(11 - et_x_0@et_player_x)
    ),
    [
        player_weights@et_player_x <= player_capacity,
        np.ones(len(players))@et_player_x == player_num,
#         et_x_0@et_player_x >= 10
    ]
)

In [308]:
et_player_prob.solve()

3.921052625744931

In [309]:
et_player_selection = [int(round(j)) for j in et_player_x.value]
et_player_selection_names = [i for i, j in zip(player_names, et_player_selection) if j == 1]
et_player_selection_names

['Kanté',
 'Milivojevic',
 'Digne',
 'Robertson',
 'van Dijk',
 'Salah',
 'Laporte',
 'Ederson',
 'Sterling',
 'Wan-Bissaka',
 'Jiménez']

In [306]:
print(set(et_player_selection_0_names) - set(et_player_selection_names))
print(set(et_player_selection_names) - set(et_player_selection_0_names))

set()
set()
